In [1]:
import zmq
import numpy as np
import time
import scipy.signal as sig
import matplotlib.pyplot as plt
import threading
from six.moves import queue

from rfsoc_qsfp_offload.overlay import Overlay
ol = Overlay(ignore_version=True)
from pynq import allocate


## Set up ZMQ socket

The address of the socket should be the computer that is connected to the RFSoC PS via ethernet or USB. Use a port that is open.

In [2]:
context = zmq.Context()
socket = context.socket(zmq.SUB)
socket.connect("tcp://10.29.234.35:50241")
# socket.connect("tcp://hay-rfsoc-003.mit.edu:55555") # connect, not bind, the PUB will bind, only 1 can bind
socket.setsockopt(zmq.SUBSCRIBE, b'') # subscribe to topic of all (needed or else it won't work)


## Initialize DAC

Send in a frequency to the DAC. So far the fastest lowest sample frequency I can send in is 614.4 Msps. 

In [3]:
DAC_TILE = 0       # DAC Tile 228
DAC_BLOCK = 0       # DAC Block 0
DAC_SAMPLE_FREQUENCY = 4915.2/8#4915.2  # MSps
DAC_PLL_FREQUENCY = 491.52   # MHz
DAC_FC = 0.0

ol.initialise_dac(tile=DAC_TILE,
                  block=DAC_BLOCK,
                  pll_freq=DAC_PLL_FREQUENCY,
                  fs=DAC_SAMPLE_FREQUENCY,
                  fc=DAC_FC
                 )

In [14]:
def fill_buff(data_queue, data_len,end_tx, up_s, dn_s):
    """This fills up a queue which will be popped off when it's to be sent to the transmitter. 
    This function also does rational resampling. There is a counter also to check if there is anything on the zmq socket.
    
    Parameters
    ----------
    data_queue : queue
        When the buffer is full the data will be added to this queue
    data_len : int
        Size of the buffer for both I/Q samples.
    end_tx : event
        This is a threading event that will end the function if set to true.
    up_s : int
        Amount of upsampling
    dn_s : int
        Amount of down sampling 
    """
    # This is a counter
    pnter =0
    time_count=1
    
    print("Fill buffer: {0}".format(end_tx.is_set()))
    
    while not end_tx.is_set():
        if time_count>0:
            pnter =0
            # set up the buffer that will be added to the queue.
            radbuff = np.empty(shape=(data_len), dtype=np.dtype('<i2'))
            time_count = 0
            
        if socket.poll(10) != 0: # check if there is a message on the socket
            time_count=0
            msg = socket.recv() # grab the message
            # Make the numpy array , assume it's interleved shorts.
            msg_data = np.frombuffer(msg, dtype=np.dtype('<i2'), count=-1)
            # HACK change the data to float to do the resampling.
            flt_data = msg_data[::2].astype(np.float32)+1j*msg_data[1::2].astype(np.float32)
            interp_data = sig.resample_poly(flt_data,up_s,dn_s)
            # Get the data back to interleaved shorts
            out_data = np.empty(interp_data.size*2,dtype=np.dtype('<i2'))
            out_data[::2] = interp_data.real.astype(np.dtype('<i2'))
            out_data[1::2] = interp_data.imag.astype(np.dtype('<i2'))
            # updated the pointers and 
            pnt_end = pnter+len(out_data)
            if pnt_end>=data_len:
                end_buff = data_len-pnter
                radbuff[pnter:] = out_data[:end_buff]
                data_queue.put(radbuff)
                nleft = len(out_data) - end_buff
                radbuff[:nleft] = out_data[end_buff:]
                pnter=nleft
            else:
                radbuff[pnter:pnt_end] = out_data
                pnter = pnt_end

        else:
            time.sleep(0.1)
            time_count+=1
            if time_count >1000:
                print("Ending because no data coming through.")
                break
    print("Buffer filling ended.")


## Run thread and tx loop

Set up the queue, buffer thread, and tx. The is cell will end gracefully with a control -c, it will end the buffering thread, reading from zmq.

In [13]:
# This is the fifo that save the samples from the buffer thread.
datafifo = queue.Queue()
# Create a threading event and set it to false.
end_tx = threading.Event()
end_tx.clear()
data_len = int(DAC_SAMPLE_FREQUENCY*1e6/16)
print(data_len)
# tx_buffer = allocate(shape=(data_len,), dtype=np.dtype('<i2'))
# Upsample by a factor of 16 based off of the gnuradio flowgraph.
up_s = 16
dn_s = 1
read_th = threading.Thread(
                target=fill_buff, args=(datafifo, data_len, end_tx,up_s,dn_s)
            )
read_th.start()
read_th.setName("Buffer Thread")


while read_th.is_alive():
    try:
        if not datafifo.empty():
            d1 = datafifo.get()
            tx_buffer[:] = d1

            print("Tx out") # size of msg

            ol.axi_dma_dac.sendchannel.transfer(tx_buffer)
            time.sleep(.25)
            ol.axi_dma_dac.sendchannel.stop()

        else:
            time.sleep(0.1)
    except KeyboardInterrupt:
        print("Ending transfer.")
    finally:
        end_tx.set()  
        
print("Finished")
ol.axi_dma_dac.sendchannel.stop()       
    
            

38400000
Fill buffer: False
Finished


ol.axi_dma_dac.sendchannel.stop()

In [15]:
datafifo = queue.Queue()
end_tx = threading.Event()
end_tx.clear()
data_len = int(DAC_SAMPLE_FREQUENCY*1e6/16)
read_th = threading.Thread(
                target=fill_buff, args=(datafifo, data_len, end_tx)
            )
read_th.start()
read_th.setName("Read Thread")

Fill buffer: False


In [3]:
from pynq import allocate
data_len = int(DAC_SAMPLE_FREQUENCY*1e6/16)
print(data_len)
tx_buffer = allocate(shape=(data_len,), dtype=np.dtype('<i2'))


76800000


RuntimeError: Allocate failed: 4294967295

In [13]:
end_tx = threading.Event()
end_tx.clear()
print(end_tx.is_set())
end_tx.set()
print(end_tx.is_set())

False
True


In [25]:
print(read_th.is_alive())
end_tx.set()
print(datafifo.empty())
blah = datafifo.get()
print(blah[:10])
print(len(blah))

False
False
[749 787 749 787 749 787 748 787 748 786]
38400000
